In [11]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import re
from datetime import datetime
import numpy as np
from configparser import ConfigParser
import unidecode
import urllib.request
import requests
import uuid

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
import requests

In [2]:
def get_all_link_spar():
    offer_page = 'https://www.spar.hu/onlineshop/'

    r = requests.get(offer_page)
    soup = bs(r.content)
    a_list = soup.body.findAll('a', {'class' : ['flyout-categories__link']})
    all_link = set()
    for a in a_list:
        if a['href'] != 'javascript:void(0)':
            ref = a['href']
            #print(ref)
            base_string = 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:'
            categ = ref[ref.find('/c/'):].replace('/c/','').replace('/','')
            all_link.add(base_string + categ)
            #all_link.append('https://www.spar.hu/onlineshop'+a['href']+'?query=*&q=*&hitsPerPage=800')
    
    return list(all_link)

In [3]:
all_link = get_all_link_spar()

In [4]:
all_link

['https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-3-1',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-11-1',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H2-1-3',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H11-10',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H8-7-1',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H17-5-3',
 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H9-5-1',
 'https://search-spar.spar-ics.c

In [5]:
fake_headers = {'Host': 'search-spar.spar-ics.com',
               'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:102.0) Gecko/20100101 Firefox/102.0',
               'Accept': 'application/json, text/javascript, */*; q=0.01',
               'Accept-Language': 'hu-HU,hu;q=0.8,en-US;q=0.5,en;q=0.3',
               'Accept-Encoding': 'gzip, deflate, br',
               'Origin': 'https://www.spar.hu',
               'Connection': 'keep-alive',
               'Referer': 'https://www.spar.hu/',
               'Sec-Fetch-Dest': 'empty',
               'Sec-Fetch-Mode': 'cors',
               'Sec-Fetch-Site': 'cross-site',
               'TE': 'trailers'}

In [6]:
url = 'https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H3-5-2'

In [7]:
response = requests.get(url, headers=fake_headers)

In [22]:
response.json()['hits']

[{'masterValues': {'category-names': 'Tejdesszertek|Tejszeletek, túró-rudi|Tejtermék, sajt, tojás, kényelmi termékek',
   'badge-icon': '/onlineshop/medias/sys_master/images/images/h0c/hab/9444906631198/chilled-v-gleges.png',
   'description': 'PÖTTY.TÚRÓ RUDI 10*30G',
   'sales-unit': 'db',
   'title': 'Pöttyös Túró Rudi natúr túródesszert 10 x 30 g (300 g)',
   'sales': '26349',
   'badge-names': 'Hűtött',
   'was-price-per-unit': '4 966.67 Ft/kg',
   'item-type': 'Product',
   'code-internal': '8796333015041',
   'price': 1490.0,
   'created-at': '1526609249891',
   'categories': ['H3_Tejtermék, sajt, tojás, kényelmi termékek:H3-5_Tejdesszertek',
    'H3-5_Tejdesszertek:H3-5-2_Tejszeletek, túró-rudi',
    'root_null:H3_Tejtermék, sajt, tojás, kényelmi termékek'],
   'best-price': 1490.0,
   'stock-status': 'inStock',
   'is-new': 'false',
   'image-url': 'https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/355820001/dt_main.jpg',
   'category-id': 'H3-5-2|1104999999|H3-5|

In [21]:
def image_download(img_url):
    print('invoked..')
    uuid_image = uuid.uuid1()
    res = requests.get(img_url)

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'jpeg':

        image_name = str(uuid_image) + '.jpg'

    if res.headers['Content-Type'][res.headers['Content-Type'].find('/')+1:] == 'plain':

        image_name = str(uuid_image) + '.png'
    
    opener = urllib.request.build_opener()
    opener.addheaders = [('Host', 'cdn1.interspar.at'), ('User-Agent' , 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:103.0) Gecko/20100101 Firefox/103.0')]
    urllib.request.install_opener(opener)

    img_path = '/Users/kuligabor/Documents/HotDealsHungary/img/' + image_name
    urllib.request.urlretrieve(img_url , img_path)
    
    return image_name

In [22]:
all_items = []
counter = 1
for url in all_link:
    response = requests.get(url, headers=fake_headers)
    print(f'crawl url: {url} done {counter} from {len(all_link)}')
    counter = counter + 1
    for hit in response.json()['hits']:
        
        item_dict = {}
        item_dict['itemId'] = hit['masterValues']['product-number']
        item_dict['itemName'] = hit['masterValues']['title']
        item_dict['itemCleanName'] = unidecode.unidecode(item_dict['itemName']).lower()
        #item_dict['imageUrl'] = hit['masterValues']['image-url']
        print(hit['masterValues']['image-url'])
        item_dict['imageUrl'] = image_download(hit['masterValues']['image-url'])
        item_dict['price'] = hit['masterValues']['price']
        item_dict['measure'] = hit['masterValues']['sales-unit']
        item_dict['salesStart'] = np.nan
        item_dict['source'] = 'spar'
        item_dict['runDate'] = datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
        item_dict['shopName'] = 'spar'
        if (hit['masterValues']['regular-price'] != hit['masterValues']['price']):
            item_dict['isSales'] = 1
        else:
            item_dict['isSales'] = 0
            
        
        all_items.append(item_dict)

df = pd.DataFrame(all_items)

crawl url: https://search-spar.spar-ics.com/fact-finder/rest/v4/search/products_lmos_hu?query=*&q=*&hitsPerPage=1000&filter=category-path:H5-3-1 done 1 from 564
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/456928002/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/427912009/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/431340003/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/456924004/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/316540009/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/342881008/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/393912003/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/393913000/dt_main.jpg
invoked..
https://cdn1.interspar.at/cachableservlets/articleImage.dam/hu/298332005/dt_mai

KeyboardInterrupt: 

In [10]:
df['isSales'].value_counts()

0    43903
1      427
Name: isSales, dtype: int64

In [11]:
df = df.fillna('N.a')

In [12]:
df.loc[~df['price'].str.isnumeric()]

AttributeError: Can only use .str accessor with string values!

In [13]:
df[df['imageUrl'].isnull()]

,itemId,itemName,itemCleanName,imageUrl,price,measure,salesStart,source,runDate,shopName,isSales


In [14]:
df.to_excel('spar.xlsx', index=False)